In [ ]:
import pandas as pd
import utils
import configs as config

clean_file_names = {
    "btm": "Portfolios Formed on Book-to-Market",
    "industry": "Industry Portfolios",
    "momentum": "Portfolios Formed on Momentum",
    "size": "Portfolios Formed on Size",
    "size_ltr": "Portfolios Formed on Size and Long-Term Reversal",
    "size_str": "Portfolios Formed on Size and Short-Term Reversal",
    "sizebtm": "Portfolios Formed on Size and Book-to-Market",
    "sizemomentum": "Portfolios Formed on Size and Momentum",
}

In [ ]:
files = utils.list_s3_files(config.BUCKET_NAME, prefix="output")

In [ ]:
results = pd.DataFrame()
for file_name in files:
    clean_file_name = file_name.replace("output/", "").replace(
        "_forecast_error.csv", ""
    )
    df = utils.read_s3_file(file_name)
    df = pd.DataFrame(df.set_index("date").mean()).T
    df.index = [clean_file_name]
    results = pd.concat([results, df])
results.index = results.index.map(clean_file_names)
overall_avg = pd.DataFrame(results.mean()).T
overall_avg.index = ["Averages Over All Datasets"]
results = pd.concat([results, overall_avg])

In [82]:
data = {
    "Dataset": [
        "Portfolios Formed on Size",
        "Portfolios Formed on Book-to-Market",
        "Portfolios Formed on Momentum",
        "Portfolios Formed on Size and Book-to-Market",
        "Industry Portfolios",
        "Portfolios Formed on Size and Momentum",
        "Portfolios Formed on Size and Short-Term Reversal",
        "Portfolios Formed on Size and Long-Term Reversal",
    ],
    "HIST": [0.0069, 0.0089, 0.0149, 0.0081, 0.0081, 0.0149, 0.0079, 0.0153],
    "SHRINK": [0.0069, 0.0080, 0.0149, 0.0081, 0.0081, 0.0149, 0.0079, 0.0153],
    "EWMA": [0.0040, 0.0042, 0.0069, 0.0042, 0.0052, 0.0069, 0.0048, 0.0093],
    "DCC-GARCH": [0.0033, 0.0035, 0.0061, 0.0036, 0.0045, 0.0061, 0.0040, 0.0081],
    "GO-GARCH": [0.0028, 0.0043, 0.0070, 0.0040, 0.0043, 0.0070, 0.0039, 0.0074],
}
df = pd.DataFrame(data).set_index("Dataset")
df.index.name = None
overall_avg = pd.DataFrame(df.mean()).T
overall_avg.index = ["Averages Over All Datasets"]
df = pd.concat([df, overall_avg])
df.columns = ["Rolling Average", "LW Shrunk", "EWMA", "DCC-GARCH", "GO-GARCH"]

In [91]:
df  # .div(df["DCC-GARCH"], axis=0)

,Rolling Average,LW Shrunk,EWMA,DCC-GARCH,GO-GARCH
Portfolios Formed on Size,0.006900,0.006900,0.004000,0.0033,0.002800
Portfolios Formed on Book-to-Market,0.008900,0.008000,0.004200,0.0035,0.004300
Portfolios Formed on Momentum,0.014900,0.014900,0.006900,0.0061,0.007000
Portfolios Formed on Size and Book-to-Market,0.008100,0.008100,0.004200,0.0036,0.004000
Industry Portfolios,0.008100,0.008100,0.005200,0.0045,0.004300
Portfolios Formed on Size and Momentum,0.014900,0.014900,0.006900,0.0061,0.007000
Portfolios Formed on Size and Short-Term Reversal,0.007900,0.007900,0.004800,0.0040,0.003900
Portfolios Formed on Size and Long-Term Reversal,0.015300,0.015300,0.009300,0.0081,0.007400
Averages Over All Datasets,0.010625,0.010513,0.005687,0.0049,0.005088


In [90]:
results.reindex(df.index)  # .div(results.reindex(df.index)["DCC-GARCH"], axis=0)

,Rolling Average,LW Shrunk,EWMA,DCC-GARCH,GO-GARCH
Portfolios Formed on Size,0.004293,0.004292,0.003539,0.002797,0.003121
Portfolios Formed on Book-to-Market,0.005530,0.005528,0.004366,0.003359,0.003601
Portfolios Formed on Momentum,0.006467,0.006465,0.004538,0.003267,0.004091
Portfolios Formed on Size and Book-to-Market,0.002014,0.002014,0.001579,0.001235,0.001221
Industry Portfolios,0.004054,0.004052,0.003404,0.002757,0.002604
Portfolios Formed on Size and Momentum,0.002336,0.002335,0.001733,0.001307,0.001424
Portfolios Formed on Size and Short-Term Reversal,0.002839,0.002839,0.002417,0.001909,0.002012
Portfolios Formed on Size and Long-Term Reversal,0.001789,0.001788,0.001443,0.001115,0.001190
Averages Over All Datasets,0.003665,0.003664,0.002877,0.002218,0.002408
